In [ ]:
import numpy as np
import sep
import matplotlib.pyplot as plt
from astropy.io import fits
%matplotlib inline
from matplotlib import rcParams

rcParams['figure.figsize'] = [10., 8.]

In [ ]:
fname = fits.util.get_testdata_filepath('hlsp_hudf12_hst_wfc3ir_udfmain_f105w_v1.0_drz.fits')
hdul = fits.open(fname)
hdul.info()

In [ ]:
data = fits.getdata(fname)
print(type(data))
print(data.shape)

In [ ]:
plt.imshow(data, cmap='gray')
plt.colorbar()

In [ ]:
data = data.byteswap().newbyteorder()

bkg = sep.Background(data)

In [ ]:
print(bkg.globalback)
print(bkg.globalrms)

In [ ]:
bkg_image = bkg.back()

In [ ]:
plt.imshow(bkg_image, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();

In [ ]:
bkg_rms = bkg.rms()

In [ ]:
plt.imshow(bkg_rms, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();

In [ ]:
data_sub = data - bkg

In [ ]:
objects = sep.extract(data_sub, 1.5, err=bkg.globalrms)

In [ ]:
len(objects)

In [ ]:
from matplotlib.patches import Ellipse

# plot background-subtracted image
fig, ax = plt.subplots()
m, s = np.mean(data_sub), np.std(data_sub)
im = ax.imshow(data_sub, interpolation='nearest', cmap='gray',
               vmin=m-s, vmax=m+s, origin='lower')

# plot an ellipse for each object
for i in range(len(objects)):
    e = Ellipse(xy=(objects['x'][i], objects['y'][i]),
                width=6*objects['a'][i],
                height=6*objects['b'][i],
                angle=objects['theta'][i] * 180. / np.pi)
    e.set_facecolor('none')
    e.set_edgecolor('red')
    ax.add_artist(e)

In [ ]:
objects.dtype.names

In [ ]:
flux, fluxerr, flag = sep.sum_circle(data_sub, objects['x'], objects['y'],
                                     3.0, err=bkg.globalrms, gain=1.0)

In [ ]:
for i in range(10):
    print("object {:d}: flux = {:f} +/- {:f}".format(i, flux[i], fluxerr[i]))

In [ ]:
print('Min:', np.min(flux))
print('Max:', np.max(flux))
print('Mean:', np.mean(flux))
print('Stdev', np.std(flux))


In [ ]:
histogram = plt.hist(flux.flatten(), bins='auto')


In [ ]:
plt.imsave("figure1.png", data)
plt.imsave("figure2.png", bkg_image)
plt.imsave("figure3.png", bkg_rms)
plt.imsave("figure4.png", data_sub)